In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
os.chdir("/content/drive/My Drive/ResnetTPU")

Mounted at /content/drive


In [0]:
import tensorflow as tf
import json
# Auth on Colab (little wrinkle: without auth, Colab will be extremely slow in accessing data from a GCS bucket, even public).
IS_COLAB_BACKEND = 'COLAB_GPU' in os.environ  # this is always set on Colab, the value is 0 or 1 depending on GPU presence
if IS_COLAB_BACKEND:
  from google.colab import auth
  auth.authenticate_user()  # Starts an Auth prompt
  if 'COLAB_TPU_ADDR' in os.environ:  # if using a TPU backend
    with tf.Session('grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])) as sess:    
      with open('<YOUR FILE>.json', 'r') as auth_info:
        # Propagate the Colab Auth so that the TPU can access your GCS buckets. This is not necessary for
        # the public bucket used in this sample but it will be once you work with your own non-public data.
        tf.contrib.cloud.configure_gcs(sess, credentials=json.load(auth_info))  # Upload the credentials to TPU

In [7]:
!pip install cloud-tpu-profiler

In [8]:
cd tpu/models/official/resnet

/content/drive/My Drive/ResnetTPU/tpu/models/official/resnet


In [9]:
%set_env PYTHONPATH=/env/python:/content/drive/My Drive/ResnetTPU/tpu/models

env: PYTHONPATH=/env/python:/content/drive/My Drive/ResnetTPU/tpu/models


In [ ]:
!echo $PYTHONPATH

In [11]:
TPU = 'grpc://' + os.environ['COLAB_TPU_ADDR']
!python2 resnet_main.py \
  --tpu=$TPU \
  --param_overrides=train_steps=936,iterations_per_loop=312,num_parallel_calls=128,train_batch_size=4096,num_train_images=1281167,num_label_classes=1000 \
  --model_dir="gs://<YOUR BUCKET NAME>/resnet-tpu-fake-data-model" \
  --mode=train \
  --profile_every_n_steps=1 \
  --tpu_profile_duration=60000

I0318 21:30:08.122473 139947819169664 resnet_main.py:553] Model params: {'dropblock_keep_prob': 0.9, 'transpose_input': True, 'use_async_checkpointing': False, 'dropblock_groups': '', 'poly_rate': 0.0, 'dropblock_size': 7, 'precision': 'bfloat16', 'base_learning_rate': 0.1, 'use_cache': True, 'image_size': 299, 'train_batch_size': 4096, 'weight_decay': 0.0001, 'num_train_images': 1281167, 'train_steps': 936, 'use_tpu': True, 'skip_host_call': False, 'eval_batch_size': 360, 'num_cores': 8, 'data_format': 'channels_last', 'resnet_depth': 50, 'num_label_classes': 1000, 'num_parallel_calls': 128, 'iterations_per_loop': 312, 'num_eval_images': 364, 'label_smoothing': 0.0, 'momentum': 0.9, 'enable_lars': False}
I0318 21:30:08.123852 139947819169664 estimator.py:201] Using config: {'_save_checkpoints_secs': None, '_session_config': graph_options {
  rewrite_options {
    disable_meta_optimizer: true
  }
}
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.41.14

In [0]:
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   


In [0]:
LOG_DIR = 'gs://<YOUR BUCKET NAME>/resnet-tpu/'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

get_ipython().system_raw('./ngrok http 6006 &')

!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://622a22e3.ngrok.io
